In [ ]:
%matplotlib inline

In [ ]:
import gc
import pathlib
import sys

In [ ]:
import joblib
import pandas as pd
from catboost import CatBoostRegressor
from catboost import Pool

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
data_dir_path = root_dir_path / "data"
processed_dir_path = data_dir_path / "processed"
train_path = processed_dir_path / "train.parquet"
models_dir_path = root_dir_path / "models"
model_path = models_dir_path / "cb_reg.joblib"
src_dir_path = root_dir_path / "src"

In [ ]:
sys.path.append(str(src_dir_path))

In [ ]:
from package.constants import *

In [ ]:
train = pd.read_parquet(train_path)

In [ ]:
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/135896
is_train = (train["date"] >= "2014-03-28") & (train["date"] < validation_start_date)
is_valid = train["date"] >= validation_start_date

In [ ]:
params = {
    "eval_metric": "RMSE",
    "max_bin": 32,
    "objective": "Poisson",
    "random_state": random_state,
    "thread_count": -1,
    "verbose": 10,
    # See https://catboost.ai/docs/concepts/faq.html#overcome-out-of-memory
    "task_type": "GPU",
    "boosting_type": "Plain",
    "max_ctr_complexity": 1,
}

In [ ]:
is_train |= is_valid

In [ ]:
train_pool = Pool(
    train.loc[is_train, features],
    train.loc[is_train, target],
    cat_features=categorical_features,
)

In [ ]:
model = CatBoostRegressor(**params)

In [ ]:
%%time
model.fit(train_pool)

In [ ]:
joblib.dump(model, model_path)